# Setting-up environment

In [1]:
!cp -r ../input/updated-code-3-2/* /kaggle/working/

In [2]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 70.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 46.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.9/466.9 kB 29.8 MB/s eta 0:00:00
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=464219 sha256=e047387456b6a50483e26f9d14ebddfff61b8636c33631afa272689de66cf3fb
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
  Created wheel for jupyter-contrib-nbextensions: filename=jupyter_contrib_nbextensions-0.7.0-py2.py3-none-any.whl size=23428781 sha256=d7498edc53200ad06fb0ec61b59e4307b249e0678ade42bb2a9ec6e7e3db966a
  Stored in directory: /root/.cache/pip/wheels/ea/cc/7d/99ef154f984

In [1]:
!pip install lightfm tqdm optuna ipykernel matplotlib implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for implicit from https://files.pythonhosted.org/packages/cd/cc/deac70cae8cc32c9885d0cd73bc66e1b3cbea36ae7080b8c83995eaf5322/implicit-0.7.2-cp310-cp310-manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 52.2 MB/s eta 0:00:0000:0100:01
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=464219 sha256=e07fa35d48822165eaf4018e6beaeec5e56b05dd28764f917c4090f102c9c2b2
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [4]:
!python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorizationImpressions_Cython_Epoch.c:1109:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorizationImpressions_Cython_Epoch.c: In functio

# Imports

In [2]:
import time
import optuna
import numpy as np
import pandas as pd
import scipy.sparse as sps
from scipy.sparse import vstack, csr_matrix
from scipy.stats import loguniform
from lightfm import LightFM
from lightfm.evaluation import auc_score, precision_at_k
import matplotlib.pyplot as plt
from multiprocessing import cpu_count

In [3]:
from Recommenders.BaseRecommender import BaseRecommender
from Recommenders.BaseMatrixFactorizationRecommender import BaseMatrixFactorizationRecommender
from Recommenders.NonPersonalizedRecommender import TopPop

#---- CF
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

#---- Matrix Factorization
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#---- CF w/ ML
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender, MultiThreadSLIM_SLIMElasticNetRecommender
from implicit.als import AlternatingLeastSquares

#---- Others
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.Recommender_utils import check_matrix

In [4]:
seed = 69
np.random.seed(seed)

# Data Import and Preprocessing

In [5]:
# Import training data
URM_path = "../input/data-books/data_train.csv"
URM_all_dataframe = pd.read_csv(filepath_or_buffer=URM_path,
                                header=0,
                                dtype={0:int, 1:int, 2:int},
                                engine='python')

URM_all_dataframe.columns = ["user_id", "item_id", "interaction"]

In [6]:
# Import target users
target_path = "../input/data-books/data_target_users_test.csv"
target_dataframe= pd.read_csv(filepath_or_buffer=target_path,
                                header=0,
                                dtype={0:int},
                                engine='python')
target_dataframe.columns = ["user_id"]
target_dataframe

,user_id
0,1
1,2
2,3
3,4
4,5
...,...
10877,13020
10878,13021
10879,13022
10880,13023


In [7]:
def preprocess_data(ratings: pd.DataFrame):
    unique_users = ratings.user_id.unique()
    unique_items = ratings.item_id.unique()

    num_users, min_user_id, max_user_id = unique_users.size, unique_users.min(), unique_users.max()
    num_items, min_item_id, max_item_id = unique_items.size, unique_items.min(), unique_items.max()

    print(num_users, min_user_id, max_user_id)
    print(num_items, min_item_id, max_item_id)

    mapping_user_id = pd.DataFrame({"mapped_user_id": np.arange(num_users), "user_id": unique_users})
    mapping_item_id = pd.DataFrame({"mapped_item_id": np.arange(num_items), "item_id": unique_items})

    ratings = pd.merge(left=ratings,
                       right=mapping_user_id,
                       how="inner",
                       on="user_id")

    ratings = pd.merge(left=ratings,
                       right=mapping_item_id,
                       how="inner",
                       on="item_id")

    return ratings

In [8]:
# Call preprocess data function
ratings = preprocess_data(URM_all_dataframe)

12638 1 13024
22222 1 22347


## Conversion to Sparse Matrix

In [9]:
URM = sps.coo_matrix((ratings.interaction.values, (ratings.mapped_user_id.values, ratings.mapped_item_id.values)))

## Data Splits

In [10]:
urm_train, urm_test = split_train_in_two_percentage_global_sample(URM, train_percentage = 0.80)
urm_train, urm_validation = split_train_in_two_percentage_global_sample(urm_train, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2630 (20.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2127 (16.8%) Users that have less than 1 test interactions


# **Custom Models**

In [11]:
class ScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1*alpha + R2*(1-alpha)

    """

    RECOMMENDER_NAME = "ScoresHybridRecommender"

    def __init__(self, URM_train, recommender_1, recommender_2):
        super(ScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2


    def fit(self, alpha=0.5):
        self.alpha = alpha

    def _compute_item_score(self, user_id_array, items_to_compute):

        # In a simple extension this could be a loop over a list of pretrained recommender objects
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)

        item_weights = item_weights_1*self.alpha + item_weights_2*(1-self.alpha)

        return item_weights

In [12]:
class LightFMCFRecommender(BaseRecommender):
    """LightFMCFRecommender"""

    RECOMMENDER_NAME = "LightFMCFRecommender"

    def __init__(self, URM_train):
        super(LightFMCFRecommender, self).__init__(URM_train)
    
    
    def fit(self, epochs = 300, user_alpha=1e-6, item_alpha = 1e-6, n_factors = 10, n_threads = 4, max_sampled=3, loss='warp', learning_schedule='adagrad'):
        
        # Let's fit a WARP model
        self.lightFM_model = LightFM(loss=loss,
                                     user_alpha=user_alpha,
                                     item_alpha=item_alpha,
                                     no_components=n_factors,
                                     max_sampled=max_sampled,
                                     learning_schedule=learning_schedule)

        self.lightFM_model = self.lightFM_model.fit(self.URM_train, 
                                       epochs=epochs,
                                       num_threads=n_threads,
                                       verbose=True)


    def _compute_item_score(self, user_id_array, items_to_compute = None):
        
        # Create a single (n_items, ) array with the item score, then copy it for every user
        items_to_compute = np.arange(self.n_items)
        
        item_scores = - np.ones((len(user_id_array), self.n_items)) * np.inf

        for user_index, user_id in enumerate(user_id_array):
            item_scores[user_index] = self.lightFM_model.predict(int(user_id), 
                                                                 items_to_compute)

        return item_scores

In [13]:
class GeneralizedLinearHybridRecommender(BaseRecommender):
    """
    This recommender merges N recommendes by weighting their ratings
    """

    RECOMMENDER_NAME = "GeneralizedLinearHybridRecommender"

    def __init__(self, URM_train, recommenders: list, verbose=True):
        self.RECOMMENDER_NAME = ''
        for recommender in recommenders:
            self.RECOMMENDER_NAME = self.RECOMMENDER_NAME + recommender.RECOMMENDER_NAME[:-11]
        self.RECOMMENDER_NAME = self.RECOMMENDER_NAME + 'HybridRecommender'

        super(GeneralizedLinearHybridRecommender, self).__init__(URM_train, verbose=verbose)

        self.recommenders = recommenders

    def fit(self, alphas=None):
        self.alphas = alphas

    def save_model(self, folder_path, file_name=None):
        pass

    def _compute_item_score(self, user_id_array, items_to_compute=None):
        result = self.alphas[0]*self.recommenders[0]._compute_item_score(user_id_array,items_to_compute)
        for index in range(1,len(self.alphas)):
            result = result + self.alphas[index]*self.recommenders[index]._compute_item_score(user_id_array,items_to_compute)
        return result

In [14]:
class FastIALSRecommender(BaseMatrixFactorizationRecommender):
    RECOMMENDER_NAME = "FastIALSRecommender"

    AVAILABLE_CONFIDENCE_SCALING = ["linear", "log"]
    
    def __init__(self, URM_train, verbose=True):
        super().__init__(URM_train, verbose=verbose)
        
    def fit(self,
            factors=20,
            regularization=1e-3,
            iterations=100,
            calculate_training_loss=False,
            num_threads=0,
            confidence_scaling='linear',
            alpha=1.0,
            epsilon=0,
            #---- Do not change
            use_native=True,
            use_cg=True,
            use_gpu=True):
        if confidence_scaling not in self.AVAILABLE_CONFIDENCE_SCALING:
           raise ValueError("Value for 'confidence_scaling' not recognized. Acceptable values are {}, provided was '{}'".format(self.AVAILABLE_CONFIDENCE_SCALING, confidence_scaling))

        self.alpha = alpha
        self.epsilon = epsilon
        self.num_factors = factors
        self.reg = regularization
        
        self.USER_factors = self._init_factors(self.n_users, False)  # don't need values, will compute them
        self.ITEM_factors = self._init_factors(self.n_items)
        
        self.recommender = AlternatingLeastSquares(factors=factors, regularization=regularization,
                                                        use_native=use_native, use_cg=use_cg, use_gpu=use_gpu,
                                                        iterations=iterations,
                                                        calculate_training_loss=calculate_training_loss,
                                                        num_threads=num_threads)
        
        self._build_confidence_matrix(confidence_scaling)
        self.recommender.fit(self.C, show_progress=self.verbose)
        
        self.USER_factors = self.recommender.user_factors.to_numpy()
        self.ITEM_factors = self.recommender.item_factors.to_numpy()
        
    
    def _linear_scaling_confidence(self):

        C = check_matrix(self.URM_train, format="csr", dtype = np.float32)
        C.data = 1.0 + self.alpha*C.data

        return C

    def _log_scaling_confidence(self):

        C = check_matrix(self.URM_train, format="csr", dtype = np.float32)
        C.data = 1.0 + self.alpha * np.log(1.0 + C.data / self.epsilon)

        return C
    
    def _build_confidence_matrix(self, confidence_scaling):

        if confidence_scaling == 'linear':
            self.C = self._linear_scaling_confidence()
        else:
            self.C = self._log_scaling_confidence()

        self.C_csc= check_matrix(self.C.copy(), format="csc", dtype = np.float32)
    
    def _init_factors(self, num_factors, assign_values=True):

        if assign_values:
            return self.num_factors**-0.5*np.random.random_sample((num_factors, self.num_factors))

        else:
            return np.empty((num_factors, self.num_factors))

# **Best Model Params**

In [15]:
ItemKNN_params = {
    'topK': 6,
    'shrink': 15,
    'similarity': 'jaccard',
    'normalize': False
}

In [16]:
RP3beta_params = {
    'alpha': 0.307953246083667, 
    'beta': 0.3073797221110665, 
    'topK': 59, 
    'normalize_similarity': True
}

In [17]:
alpha_itemknn_rp3beta=0.7381515719042592

In [18]:
UserKNN_params = {
    'topK': 470,
    'shrink': 0,
    'similarity': 'cosine',
    'normalize': True
}

In [19]:
LightFM_params = {
                  'n_factors': 482,
                  'max_sampled': 5,
                  'user_alpha': 0.00023989649900734266,
                  'item_alpha': 9.740651135253414e-05
                 }

In [20]:
SLIM_params = {
    'l1_ratio': 0.013752256221164005,
    'alpha': 0.0031943927190071775,
    'topK': 622
}

In [21]:
NMF_params = {
    'l1_ratio': 0.005734775635120469,
    'num_factors': 134,
    'beta_loss': 'frobenius',
    'init_type': 'nndsvda',
    'solver': 'multiplicative_update'
}

In [22]:
IALS_params = {
    'confidence_scaling': 'log',
    'epsilon': 0.11624415533664904,
    'factors': 116,
    'regularization': 0.005454427904241962,
    'alpha': 1.7221339971074425
}

# Pipeline Model
The goal of this approach is to first use some models to create an enriched URM, then use this URM as input for another model.

### Phase 1 (Enriching)
Pass the URM through:
1. Hybrid ItemKNN + RP3beta
2. UserKNN

### Phase 2 (Fitting + Prediction)
1. Tune a Linear Hybrid model:
        (1. ItemKNN + RP3beta)
        2. LightFM
        3. SLIM
        4. IALS
2. Profit ?

## URM Enriching

In [75]:
userKNNCF = UserKNNCFRecommender(urm_train)
userKNNCF.fit(**UserKNN_params)

UserKNNCFRecommender: URM Detected 462 ( 3.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 344 ( 1.5%) items with no interactions.
Similarity column 12638 (100.0%), 6400.47 column/sec. Elapsed time 1.97 sec


In [23]:
itemKNNCF = ItemKNNCFRecommender(urm_train)
itemKNNCF.fit(**ItemKNN_params)

ItemKNNCFRecommender: URM Detected 462 ( 3.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 344 ( 1.5%) items with no interactions.
Similarity column 22222 (100.0%), 5163.96 column/sec. Elapsed time 4.30 sec


In [24]:
rp3beta = RP3betaRecommender(urm_train)
rp3beta.fit(**RP3beta_params)

RP3betaRecommender: URM Detected 462 ( 3.7%) users with no interactions.
RP3betaRecommender: URM Detected 344 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 1805.84 column/sec. Elapsed time 12.31 sec


In [25]:
new_similarity = (1 - alpha_itemknn_rp3beta) * itemKNNCF.W_sparse + alpha_itemknn_rp3beta * rp3beta.W_sparse
    
itemKNN_rp3beta = ItemKNNCustomSimilarityRecommender(urm_train)
itemKNN_rp3beta.fit(new_similarity)

ItemKNNCustomSimilarityRecommender: URM Detected 462 ( 3.7%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 344 ( 1.5%) items with no interactions.


### Building new URM (From ItemKNN & RP3beta)

In [26]:
def batch_compute_scores(user_ids, model, n_items):
    items_to_compute = np.arange(n_items)
    new_ratings_batch = model._compute_item_score(user_ids, items_to_compute)
    return csr_matrix(new_ratings_batch)  # Convert each batch to CSR format immediately

In [27]:
def enrich_urm(model, ratings, batch_size=100):
    n_users = ratings.mapped_user_id.nunique()
    n_items = model.n_items
    user_id_array = np.array(ratings.mapped_user_id)

    # Create an empty list to store batch results
    all_new_ratings = []

    # Process in batches
    for start_idx in range(0, n_users, batch_size):
        end_idx = min(start_idx + batch_size, n_users)
        batch_user_ids = user_id_array[start_idx:end_idx]
        new_ratings_batch = batch_compute_scores(batch_user_ids, model, n_items)
        all_new_ratings.append(new_ratings_batch)

    # Combine all the batch results into one CSR matrix
    final_new_ratings_csr = vstack(all_new_ratings, format='csr')
    return final_new_ratings_csr

In [28]:
batch_size = 500
urm_train_enriched = enrich_urm(itemKNN_rp3beta, ratings, batch_size=batch_size)

In [172]:
(urm_train_enriched.nnz, urm_train.nnz)

(18780870, 306387)

In [55]:
bol = urm_train_enriched[0] / urm_train_enriched[0].toarray().max()
bol.toarray()[0][300:400]

array([0.06483463, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.06637032, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.08682911, 0.        ,
       0.16364757, 0.        , 0.        , 0.07550569, 0.0600001 ,
       0.        , 0.        , 0.07269746, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.06535504, 0.        ,
       0.        , 0.        , 0.06157309, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.24703484, 0.        ,
       0.        , 0.        , 0.19644647, 0.        , 0.        ,
       0.        , 0.06180081, 0.12979469, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [126]:
urm_train_enriched = urm_train_enriched.toarray()
urm_train_enriched[urm_train_enriched > 1] = 1
urm_train_enriched = csr_matrix(urm_train_enriched)

In [173]:
urm_train_enriched = urm_train_enriched / urm_train_enriched.toarray().max()

In [199]:
def sparsify_matrix(matrix, percentile):
    # Convert to dense array if it's in sparse format
    if isinstance(matrix, csr_matrix):
        matrix = matrix.toarray()

    # Calculate the threshold
    threshold = np.percentile(matrix, percentile)

    # Set values below the threshold to zero
    matrix[matrix < threshold] = 0
    matrix[matrix >= threshold] = 0.23  # Low confidence

    # Convert back to sparse CSR format
    return csr_matrix(matrix)

In [200]:
percentile = 99.999
urm_train_enriched_sparse = sparsify_matrix(urm_train_enriched, percentile=percentile)

In [175]:
(urm_train_enriched.nnz, urm_train_enriched_sparse.nnz, urm_train.nnz)

(18780870, 326, 306387)

In [201]:
urm_train_enriched_sparse_add = urm_train_enriched_sparse + urm_train

In [159]:
enriched_delta = urm_train_enriched_sparse - urm_train
enriched_delta.nnz

2809

In [202]:
userKNNCF = UserKNNCFRecommender(urm_train_enriched_sparse_add)
userKNNCF.fit(**UserKNN_params)

UserKNNCFRecommender: URM Detected 412 ( 3.3%) users with no interactions.
UserKNNCFRecommender: URM Detected 344 ( 1.5%) items with no interactions.
Similarity column 12638 (100.0%), 6740.13 column/sec. Elapsed time 1.88 sec


In [187]:
userKNNCF_t = UserKNNCFRecommender(urm_train)
userKNNCF_t.fit(**UserKNN_params)

UserKNNCFRecommender: URM Detected 462 ( 3.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 344 ( 1.5%) items with no interactions.
Similarity column 12638 (100.0%), 6505.13 column/sec. Elapsed time 1.94 sec


In [177]:
itemKNNCF_E = ItemKNNCFRecommender(urm_train_enriched_sparse_add)
itemKNNCF_E.fit(**ItemKNN_params)

ItemKNNCFRecommender: URM Detected 454 ( 3.6%) users with no interactions.
ItemKNNCFRecommender: URM Detected 344 ( 1.5%) items with no interactions.
Similarity column 22222 (100.0%), 6427.40 column/sec. Elapsed time 3.46 sec


In [161]:
rp3beta_E = RP3betaRecommender(urm_train_enriched_sparse)
rp3beta_E.fit(**RP3beta_params)

RP3betaRecommender: URM Detected 412 ( 3.3%) users with no interactions.
RP3betaRecommender: URM Detected 344 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 3474.07 column/sec. Elapsed time 6.40 sec


In [162]:
new_similarity = (1 - alpha_itemknn_rp3beta) * itemKNNCF_E.W_sparse + alpha_itemknn_rp3beta * rp3beta_E.W_sparse
    
itemKNN_rp3beta_E = ItemKNNCustomSimilarityRecommender(urm_train_enriched_sparse)
itemKNN_rp3beta_E.fit(new_similarity)

ItemKNNCustomSimilarityRecommender: URM Detected 412 ( 3.3%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 344 ( 1.5%) items with no interactions.


In [203]:
result_df, _ = evaluator_validation.evaluateRecommender(userKNNCF)
result_df

EvaluatorHoldout: Processed 10008 (100.0%) in 8.97 sec. Users per second: 1116


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.051898,0.10542,0.094843,0.023316,0.047214,0.166761,0.089593,0.067087,0.368405,0.196447,...,0.791897,0.291739,0.791897,0.010816,7.623156,0.985433,0.030782,0.58648,3.722943,0.308311


In [189]:
result_df, _ = evaluator_validation.evaluateRecommender(userKNNCF_t)
result_df

EvaluatorHoldout: Processed 10008 (100.0%) in 8.86 sec. Users per second: 1129


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.052028,0.105997,0.095424,0.023344,0.047227,0.167043,0.089793,0.06734,0.369604,0.196727,...,0.791897,0.292689,0.791897,0.0104,7.602932,0.98541,0.030187,0.58611,3.661552,0.311509


In [41]:
urm_train_enriched = enrich_urm(userKNNCF, ratings, batch_size=batch_size)

In [42]:
urm_train_enriched.nnz

242014271

# Tuning augmentation

In [33]:
def parametric_sparsify_matrix(matrix, percentile, confidence):
    # Convert to dense array if it's in sparse format
    if isinstance(matrix, csr_matrix):
        matrix = matrix.toarray()

    # Calculate the threshold
    threshold = np.percentile(matrix, percentile)

    # Set values below the threshold to zero
    matrix[matrix < threshold] = 0
    matrix[matrix >= threshold] *= confidence

    # Convert back to sparse CSR format
    return csr_matrix(matrix)

In [37]:
def fix_increase_enriched_urm(original_urm, user_knn_output, max_interaction_increase):
    # Convert the original URM to LIL format for efficient modification
    enriched_urm = original_urm.tolil()

    for user_idx in range(user_knn_output.shape[0]):
        # Get non-zero interactions from UserKNN output
        user_knn_interactions = user_knn_output.getrow(user_idx).tocoo()

        # Sort interactions by confidence value
        sorted_indices = np.argsort(user_knn_interactions.data)[::-1]
        
        # Calculate the cap for additional interactions
        interaction_cap = int(original_urm.getrow(user_idx).count_nonzero() * max_interaction_increase)
        
        # Keep top interactions up to the maximum cap
        for rank, idx in enumerate(sorted_indices):
            if rank >= interaction_cap:
                break
            item_idx = user_knn_interactions.col[idx]
            enriched_urm[user_idx, item_idx] = max(enriched_urm[user_idx, item_idx], user_knn_interactions.data[idx])

    # Convert back to CSR format before returning
    return enriched_urm.tocsr()

In [30]:
class SaveResults(object):

    def __init__(self):
        self.results_df = pd.DataFrame(columns=["result"])

    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]

        # Create a DataFrame from the current trial's results
        trial_df = pd.DataFrame([hyperparam_dict])

        # Use concat instead of append
        self.results_df = pd.concat([self.results_df, trial_df], ignore_index=True)

In [38]:
def objective_function_sparsify(optuna_trial):
    confidence = optuna_trial.suggest_float("confidence", 0, 1)
    percentile = optuna_trial.suggest_float("percentile", 99.9, 100)
    max_interaction_increase = optuna_trial.suggest_float("max_interaction_increase", 1, 2)
    
    urm_train_enriched_sparse = parametric_sparsify_matrix(urm_train_enriched, percentile=percentile, confidence=confidence)
    urm_train_enriched_capped = fix_increase_enriched_urm(urm_train, urm_train_enriched_sparse, max_interaction_increase)
    
    #urm_train_enriched_sparse_add = urm_train_enriched_sparse + urm_train
    
    recommender_instance = UserKNNCFRecommender(urm_train_enriched_capped)
    recommender_instance.fit(**UserKNN_params)

    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)

    return result_df.loc[10]["MAP"]

In [39]:
optuna_study_sparsify = optuna.create_study(direction="maximize")

save_results_sparsify = SaveResults()

optuna_study_sparsify.optimize(objective_function_sparsify,
                      callbacks=[save_results_sparsify],
                      n_trials = 300)

[I 2023-12-05 22:10:18,639] A new study created in memory with name: no-name-3ad7cc35-9772-4631-a54d-559ed96791ef


UserKNNCFRecommender: URM Detected 462 ( 3.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 344 ( 1.5%) items with no interactions.
Similarity column 12638 (100.0%), 4981.11 column/sec. Elapsed time 2.54 sec
EvaluatorHoldout: Processed 10008 (100.0%) in 12.90 sec. Users per second: 776


[I 2023-12-05 22:10:46,604] Trial 0 finished with value: 0.02329941998781933 and parameters: {'confidence': 0.845229642574217, 'percentile': 99.98816302865197, 'max_interaction_increase': 1.896680983218175}. Best is trial 0 with value: 0.02329941998781933.


UserKNNCFRecommender: URM Detected 462 ( 3.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 344 ( 1.5%) items with no interactions.
Similarity column 12638 (100.0%), 4055.62 column/sec. Elapsed time 3.12 sec
EvaluatorHoldout: Processed 10008 (100.0%) in 13.61 sec. Users per second: 735


[I 2023-12-05 22:11:16,110] Trial 1 finished with value: 0.022757968228591774 and parameters: {'confidence': 0.6250567845792284, 'percentile': 99.93136335488335, 'max_interaction_increase': 1.8022666970939425}. Best is trial 0 with value: 0.02329941998781933.


...

[I 2023-12-06 00:21:07,583] Trial 297 finished with value: 0.023301216169921264 and parameters: {'confidence': 0.6398489776521092, 'percentile': 99.98335733199897, 'max_interaction_increase': 1.035365762657075}. Best is trial 241 with value: 0.023457131120500446.


UserKNNCFRecommender: URM Detected 462 ( 3.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 344 ( 1.5%) items with no interactions.
Similarity column 12638 (100.0%), 4962.23 column/sec. Elapsed time 2.55 sec
EvaluatorHoldout: Processed 10008 (100.0%) in 12.33 sec. Users per second: 812


[I 2023-12-06 00:21:34,020] Trial 298 finished with value: 0.023353079441208915 and parameters: {'confidence': 0.6523251543920354, 'percentile': 99.97827742076846, 'max_interaction_increase': 1.0465063712932139}. Best is trial 241 with value: 0.023457131120500446.


UserKNNCFRecommender: URM Detected 462 ( 3.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 344 ( 1.5%) items with no interactions.
Similarity column 12638 (100.0%), 5116.70 column/sec. Elapsed time 2.47 sec
EvaluatorHoldout: Processed 10008 (100.0%) in 12.57 sec. Users per second: 796


[I 2023-12-06 00:22:00,336] Trial 299 finished with value: 0.023294642064126528 and parameters: {'confidence': 0.6272970692435809, 'percentile': 99.98632693319776, 'max_interaction_increase': 1.0588596964716366}. Best is trial 241 with value: 0.023457131120500446.


**0.023344**


**0.023457131120500446**

In [ ]:
[I 2023-12-06 00:22:00,336] Trial 299 finished with value: 0.023294642064126528 and parameters: {'confidence': 0.6272970692435809, 'percentile': 99.98632693319776, 'max_interaction_increase': 1.0588596964716366}. Best is trial 241 with value: 0.023457131120500446.

In [ ]:
pruned_trials = [t for t in optuna_study_sparsify.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in optuna_study_sparsify.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print("  Number of finished trials: ", len(optuna_study_sparsify.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
print("  Value Validation: ", optuna_study_sparsify.best_trial.value)

print("Best params:")
print(optuna_study_sparsify.best_trial.params)